# A Combined end-to-end training model for NLP classify problem

In [ ]:
import numpy as np
import random
import threading
import warnings
import os
import sys
from datetime import datetime
import nltk
sys.path.append('../common/')

from train_log import train_log

#os.environ["CUDA_VISIBLE_DEVICES"]="1,2" 
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.FATAL)

#Datafile, json format
FILE_NAME = 'News_Category_Dataset_v2.json'

#import glove2word2vec from gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('C:/Users/Wangdi/Documents/Notebook/GloVe/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
#print(word2vec_glove_file)
glove2word2vec(glove_file, word2vec_glove_file)

model = KeyedVectors.load_word2vec_format(word2vec_glove_file)
    
import NewsCategoryData as ncd 
from NewsCategoryData import NewsCategoryTrainTestSet
from NewsCategoryData import LABEL_LIST
import Run_Prediction as rp


## Train the out-of-vocabury words with the close words in the dataset


In [ ]:
batch_size = 64
max_length = 100
data = NewsCategoryTrainTestSet(batch_size=batch_size,max_length=max_length)
word_matrix_file = "naive_bayes_word_matrix_ver1.csv"
word_matrix, word_list = rp.read_naive_bayes_word_vector(word_matrix_file)

def find_window(word, title, window_size = 2):
    '''
    word: is the word you are looking for.
    title: is a word list which contains the given word
    window_size: decide how much word you are going to return, 
    for example 2 means only return the left and right words, 
    3 means return the left 2 and right 2 words. 

    '''
    close_word_list = []
    i = 0 
    for i in range(len(title)):
        if title[i].lower() == word:
            if i > 0:
                close_word_list.extend(title[max(i-window_size+1,0):i])
            if i < len(title)-1:
                close_word_list.extend(title[i+1:min(i+window_size,len(title))])
    return close_word_list

unknown_word = {}
for title in data.train_data:
    for word in title:
        if model.vocab.get(word.lower(),"NaN") == "NaN":
            if word.lower() in unknown_word:
                unknown_word[word.lower()].append(title)
            else:
                unknown_word[word.lower()] = []
                unknown_word[word.lower()].append(title)

close_word_dict = {}
for word in unknown_word:
    close_word_dict[word] = []
    for title in unknown_word[word]:
        close_word_list = find_window(word, title, window_size = 3)
        close_word_dict[word].extend(close_word_list)
      
    
word_vector_dic = {}
for word in close_word_dict:
    word_vector_dic[word] =np.zeros(100)
    for close_word in close_word_dict[word]:
        if model.vocab.get(close_word.lower(),"NaN") != "NaN":
            word_vector_dic[word] = np.add(word_vector_dic[word],model[close_word.lower()])/2

In [ ]:
def getvector(word, model):
    if model.vocab.get(word.lower(),"NaN") == "NaN":
        lst = nltk.stem.LancasterStemmer()
        if model.vocab.get(lst.stem(word).lower(),"NaN") == "NaN":
            if word[-1] == "s" and model.vocab.get(word[:-1].lower(),"NaN") != "NaN":
                return model[word[:-1].lower()]
            elif word.lower() in word_vector_dic:
                return word_vector_dic[word.lower()]
            else:
                return model["unk"]
        else:
            return model[lst.stem(word).lower()]
    else:
        return model[word.lower()]

def word2vector(reviews, model, max_length=1000):
    vector_data = np.zeros((len(reviews), max_length,100))
    x_length = []
    i = 0
    for review in reviews:
        j = 0 
        if len(review) > max_length:
            print("The length of the reviews is %dwhich is larger than max_length (%d)"%(len(review),max_length))
            print(review)
            print("-"*10)
        for word in review:
            vector_data[i,j] = getvector(word,model)
            if str(vector_data[i,j,0])=='nan':
                print(word)
                break
            j += 1
        x_length.append(j)
        i += 1
    return vector_data, np.asarray(x_length)

In [ ]:
state_size = 100
n_classes = 41
word2vector_len = 100  

timestamp = "{0:%Y-%m-%dT%H-%M-%S/}".format(datetime.now())

ckpt_dir = './ckpt_save_combine_model/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
log_dir = './log_combine_model/log_'+timestamp
#log_dir = 'log_2021-08-26T17-12-04'
#log = train_log(log_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

## Define a function to build a single-layer recursive neural network

x1->x2->x3->final_state->y

In [ ]:
def create_simple_rnn(x_rnn, x_length, y_, batch_size, state_size):
    '''create a simple rnn structure
    input: 
    x_rnn: tensor placeholder for x input
    x_length: tensor placeholder for the dynamic_rnn paramter sequence_length 
    y_: tensor placeholder for y lable
    batch_size: define the batch size 
    return
    y: tenfor for logit output
    '''
    with tf.variable_scope('rnn'):
        init_state = tf.zeros([batch_size, state_size], name="init_state")     
        cell = tf.contrib.rnn.BasicRNNCell(state_size, name = "cell")          
        rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, inputs = x_rnn, 
                                                     dtype=tf.float32,
                                                     initial_state=init_state,
                                                     sequence_length = x_length)
        print("rnn/rnn_outputs", rnn_outputs.shape)
        print("rnn/final_state", final_state.shape)

    with tf.variable_scope('fclayer'):
        w1 = tf.Variable(tf.random_uniform([state_size, int(state_size*7)],0,1.0),name="w1")
        b1= tf.Variable(tf.zeros([int(state_size*7)]),name="b1")
        hidden = tf.tanh(tf.add(tf.matmul(final_state,w1), b1),name="hidden")
        print("fclayer/hidden", hidden.shape)

    with tf.variable_scope('rnn_output'):
        w2 = tf.Variable(tf.random_uniform([int(state_size*7), n_classes],0,1.0),name="w2")
        b2= tf.Variable(tf.zeros([n_classes]),name="b2")
        y = tf.add(tf.matmul(hidden, w2), b2,name="y")
        print("rnn_output/y", y.shape)
    return y


## Define a function to build a single-layer recursive neural network and use all the rnn output for cassify

In [ ]:
def create_simple_rnn_2(x_rnn, x_length, y_, batch_size, state_size):
    '''create a simple rnn structure
    input: 
    x_rnn: tensor placeholder for x input
    x_length: tensor placeholder for the dynamic_rnn paramter sequence_length 
    y_: tensor placeholder for y lable
    batch_size: define the batch size 
    return
    y: tenfor for logit output
    '''
    num_of_neurals = 3
    with tf.variable_scope('rnn'):
        #init_state = tf.zeros([batch_size, state_size], name="init_state")     
        cell = tf.contrib.rnn.BasicRNNCell(state_size, name = "cell")          
        rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, inputs = x_rnn, 
                                                     dtype=tf.float32,
                                                     #initial_state=init_state,
                                                     sequence_length = x_length)
        #rnn_outputs = tf.contrib.layers.flatten(rnn_outputs)
        rnn_outputs=tf.reshape(rnn_outputs,[rnn_outputs.shape[0],rnn_outputs.shape[1],rnn_outputs.shape[2],1])
        print("rnn/rnn_outputs", rnn_outputs.shape)
        print("rnn/final_state", final_state.shape)

    with tf.variable_scope('fclayer'):
        stddev = 1. / tf.sqrt(tf.cast(32, tf.float32))
        kernel = tf.Variable(tf.random_uniform([8, 8, rnn_outputs.get_shape()[-1], 32], 
                                               minval=-stddev, maxval=stddev),
                             name='kernel')
        hidden = tf.nn.conv2d(rnn_outputs, kernel, [1, 1, 1, 1], padding='VALID', name='conv')
        bias = tf.zeros([32], name = 'bias')
        hidden = tf.nn.bias_add(hidden, bias)
        hidden = tf.contrib.layers.flatten(hidden)
        print("fclayer/hidden", hidden.shape)

    with tf.variable_scope('rnn_output'):
        w2 = tf.Variable(tf.random_uniform([hidden.shape[1], n_classes],0,1.0),name="w2")
        b2= tf.Variable(tf.zeros([n_classes]),name="b2")
        y = tf.add(tf.matmul(hidden, w2), b2,name="y")
        print("rnn_output/y", y.shape)
        tf.summary.histogram("rnn_output/w2",w2)
    return y


## Define a function to build a multi-layer recursive neural network

In [ ]:
def create_multilayer_rnn(x_rnn, x_length, y_, batch_size, state_size):
    '''create a multi-layer rnn structure
    input: 
    x_rnn: tensor placeholder for x input
    x_length: tensor placeholder for the dynamic_rnn paramter sequence_length 
    y_: tensor placeholder for y lable
    batch_size: define the batch size 
    return
    y: tenfor for logit output
    '''
    with tf.variable_scope('rnn'):
        #init_state = tf.zeros([batch_size, state_size], name="init_state")
        cells = [tf.contrib.rnn.BasicLSTMCell(state_size, forget_bias = 0, name = "cell") for _ in range(3)]
        staked_rnn_cells = tf.nn.rnn_cell.MultiRNNCell(cells)
        rnn_outputs, final_state = tf.nn.dynamic_rnn(staked_rnn_cells,
                                                     inputs = x_rnn, 
                                                     dtype=tf.float32,
                                                     initial_state=init_state,
                                                     sequence_length = x_length)
        #rnn_outputs = tf.transpose(rnn_outputs,[1,0,2])
        print("rnn/rnn_outputs", rnn_outputs.shape)
        print("rnn/final_state", final_state)

    with tf.variable_scope('fclayer'):
        w1 = tf.Variable(tf.random_uniform([state_size*2, int(state_size*9)],0,1.0),name="w1")
        b1= tf.Variable(tf.zeros([int(state_size*9)]),name="b1")
        hidden = tf.nn.relu(tf.add(tf.matmul(tf.concat([final_state[2].c,
                                                    final_state[2].h],axis=1),w1), b1),name="hidden")
        print("fclayer/hidden", hidden.shape)

    with tf.variable_scope('rnn_output'):
        w2 = tf.Variable(tf.random_uniform([int(state_size*9), n_classes],0,1.0),name="w2")
        b2= tf.Variable(tf.zeros([n_classes]),name="b2")
        y = tf.add(tf.matmul(hidden, w2), b2,name="y")
        print("rnn_output/y", y.shape)
    return y

## Build the recursive neural network structure based on the rnn_type input

In [ ]:
rnn_type = 'Single-layer-2' 

graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('input'):
        x_rnn = tf.placeholder(tf.float32, [batch_size, max_length, word2vector_len], name="x_rnn")
        x_length = tf.placeholder(tf.int32, [batch_size], name="x_length")
        y_ = tf.placeholder(tf.int64, [batch_size],  name="y_")
        print('input/x_rnn',x_rnn.shape)
        print('input/y_',y_.shape)    
    
    if rnn_type == 'Single-layer' :
        y = create_simple_rnn(x_rnn, x_length, y_, batch_size, state_size)
    elif rnn_type == 'Multi-layer' :
        y =create_multilayer_rnn(x_rnn, x_length, y_, batch_size, state_size)
    elif rnn_type == 'Single-layer-2' :
        y = create_simple_rnn_2(x_rnn, x_length, y_, batch_size, state_size)
        
    with tf.variable_scope('loss'):
        cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                      labels=y_, 
                                                                                      name="cross_entropy_loss"))
        tf.summary.scalar('loss/cross_entropy', cross_entropy)

    with tf.variable_scope('Prediction'):
        prediction = tf.argmax(tf.nn.softmax(y),1, name="prediction")
        cal_accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction,y_), tf.float32),name="cal_accuracy")
        tf.summary.scalar('Prediction/cal_accuracy', cal_accuracy)

    with tf.variable_scope('Train'):
        #optimizer = tf.train.GradientDescentOptimizer(0.00001)
        optimizer = tf.train.AdamOptimizer(0.0001)
        
        #grad_w_1 = tf.reduce_mean(tf.gradients(cross_entropy, [graph.get_tensor_by_name('fclayer/w1:0')]))
        grad_w_2 =  tf.reduce_mean(tf.gradients(cross_entropy, [graph.get_tensor_by_name('rnn_output/w2:0')]))
        train_step = optimizer.minimize(cross_entropy)
        #tf.summary.scalar('Train/grad_w_1',grad_w_1)
        tf.summary.scalar('Train/grad_w_2',grad_w_2)

    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir)

## Use multi-layer rnn

In [ ]:
graph = tf.Graph()
if rnn_type == 'Single-layer RNN': 
    with graph.as_default():
        with tf.variable_scope('input'):
            x_rnn = tf.placeholder(tf.float32, [None, max_length, word2vector_len], name="x_rnn")
            x_naive_bayes = tf.placeholder(tf.float32, [None, n_classes], name="x_naive_bayes")
            x_length = tf.placeholder(tf.int32, [None], name="x_length")
            y_ = tf.placeholder(tf.int64, [None],  name="y_")

            #RNN的初始化状态，全设为零。注意state是与input保持一致，接下来会有concat操作，所以这里要有batch的维度。即每个样本都要有隐层状态
            init_state = tf.zeros([batch_size, state_size], name="init_state")

        with tf.variable_scope('rnn'):
            #定义rnn_cell的权重参数，
            #cell = tf.contrib.rnn.BasicRNNCell(state_size, name = "cell")
            cell = tf.contrib.rnn.BasicLSTMCell(state_size, forget_bias = 0, name = "cell")
            rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, inputs = x_rnn, 
                                                         dtype=tf.float32,
                                                         #initial_state=init_state,
                                                         sequence_length = x_length)
            print("rnn/final_state", final_state)

        with tf.variable_scope('fclayer'):
            w1 = tf.Variable(tf.random_uniform([state_size*2, int(state_size*9)],0,1.0),name="w1")
            b1= tf.Variable(tf.zeros([int(state_size*9)]),name="b1")
            hidden = tf.tanh(tf.add(tf.matmul(tf.concat([final_state[0],final_state[1]],axis=1), w1), b1),name="hidden")
            print("fclayer/hidden", hidden.shape)
            tf.summary.histogram('w1', w1)

        with tf.variable_scope('rnn_output'):
            w2 = tf.Variable(tf.random_uniform([int(state_size*9), n_classes],0,1.0),name="w2")
            b2= tf.Variable(tf.zeros([n_classes]),name="b2")
            y_rnn_logits = tf.add(tf.matmul(hidden, w2), b2,name="y_rnn_logits")
            y_rnn = tf.nn.softmax(y_rnn_logits, name="y_rnn")
            print("rnn_output/y_rnn", y_rnn.shape)
            tf.summary.histogram('w2', w2)

        with tf.variable_scope('combine_output'):   
            w = tf.Variable(tf.random_uniform([n_classes*2, n_classes],0,1.0),name="w")
            b= tf.Variable(tf.zeros([n_classes]),name="b")
            y = tf.add(tf.matmul(tf.concat([y_rnn,x_naive_bayes],axis=1), w), b,name="y")
            print("combine_output/y",y.shape)

        with tf.variable_scope('loss'):
            cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                          labels=y_, 
                                                                                          name="cross_entropy_loss"))
            tf.summary.scalar('cross_entropy_loss', cross_entropy)
            #cross_entropy_3 = tf.cast(tf.one_hot(y_-1,n_classes),tf.float32)*tf.log(tf.nn.softmax(y))
            
        with tf.variable_scope('Prediction'):
            prediction = tf.argmax(tf.nn.softmax(y),1, name="prediction")
            cal_accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction,y_), tf.float32),name="cal_accuracy")
            tf.summary.scalar('cal_accuracy', cal_accuracy)

        with tf.variable_scope('Train'):
            optimizer = tf.train.GradientDescentOptimizer(0.0001)
            #optimizer = tf.train.AdamOptimizer(0.00001)
            grad_w_1 = tf.reduce_mean(tf.gradients(cross_entropy, [w1]))
            grad_w_2 =  tf.reduce_mean(tf.gradients(cross_entropy, [w2]))
            train_step = optimizer.minimize(cross_entropy)
            tf.summary.scalar('Train/grad_w_1',grad_w_1)
            tf.summary.scalar('Train/grad_w_2',grad_w_2)
            
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(log_dir)

## Run the model for the test dataset
Output the accuracy and the loss of the test dataset

In [ ]:
CONTINUE = 0
start_step = 0

with graph.as_default():
    saver = tf.train.Saver(max_to_keep = 1)
    max_output_count = 0

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint(ckpt_dir)
            saver.restore(sess,model_file)
        threads = tf.train.start_queue_runners(sess, coord)
        try:
            i = start_step
            avg_accuracy = 0
            avg_loss = 0
            j = 0
            while i < 50000:    
                batch_data, batch_label = data.batch_train_set() 
                batch_data_vec, data_length = word2vector(batch_data, model,max_length)
                naive_bayes_input = rp.naive_bayes_model(batch_data,word_matrix, word_list)
                summary, prediction_result, label , loss, accuracy, _ = sess.run([merged, prediction, y_, cross_entropy, cal_accuracy, train_step],
                                                    feed_dict={x_rnn:batch_data_vec, 
                                                               #x_naive_bayes:naive_bayes_input,
                                                               y_:batch_label, 
                                                               x_length:data_length})
                

                avg_accuracy += accuracy
                avg_loss += loss
                i += 1
                #print(".",end="")
                if i%1000 == 0:
                    print("-----------%d-----------"%i)
                    print("Accuracy at %d: %.2f%%"%(i,avg_accuracy/10))
                    print("Loss at %d：%.4f"%(i, avg_loss/1000))
                    print('----------label---------')
                    print(label)
                    print('----------prediction---------')
                    print(prediction_result)
                    saver.save(sess,ckpt_dir+'news_category',global_step=i)
                    train_writer.add_summary(summary, i)
                    avg_accuracy = 0
                    avg_loss = 0

        except tf.errors.OutOfRangeError:
            print('done')
        finally:
            coord.request_stop()
            coord.join(threads)
            print('quit')

In [ ]:
with graph.as_default():
    saver = tf.train.Saver(max_to_keep = 1)
    with tf.Session() as sess:
        model_file=tf.train.latest_checkpoint(ckpt_dir)
        saver.restore(sess,model_file)
        total_accuracy = 0
        total_loss = 0
        #print(int(data.test_size/data.batch_size)+1)
        for i in range(int(data.test_size/data.batch_size)+1):
            batch_data, batch_label = data.batch_test_set()
            batch_data_vec, data_length = word2vector(batch_data, model,max_length)
            #naive_bayes_input = rp.naive_bayes_model(batch_data,word_matrix, word_list)
            y, loss, accuracy = sess.run([prediction, cross_entropy, cal_accuracy],
                                                feed_dict={x_rnn:batch_data_vec, 
                                                           #x_naive_bayes:naive_bayes_input,
                                                           y_:batch_label, 
                                                           x_length:data_length})

            total_accuracy += accuracy
            total_loss += loss
            #print(i,accuracy,loss)

        total_accuracy = total_accuracy/i
        total_loss = total_loss/i
        print("Test set accuracy: %.2f %%"%(total_accuracy*100))
        print("Test set loss: %.2f "%(total_loss))
    

## Run the model for the train dataset
Output the accuracy and the loss based on the whole train dataset

In [ ]:
with graph.as_default():
    with tf.Session() as sess:
        model_file=tf.train.latest_checkpoint(ckpt_dir)
        saver.restore(sess,model_file)
        total_accuracy = 0
        total_loss = 0
        #print(int(data.test_size/data.batch_size)+1)
        for i in range(int(data.train_size/data.batch_size)+1):
            batch_data, batch_label = data.batch_train_set()  
            batch_data_vec, data_length = word2vector(batch_data, model,max_length)
            naive_bayes_input = rp.naive_bayes_model(batch_data,word_matrix, word_list)
            y, loss, accuracy = sess.run([prediction, cross_entropy, cal_accuracy],
                                                feed_dict={x_rnn:batch_data_vec, 
                                                           x_naive_bayes:naive_bayes_input,
                                                           y_:batch_label, 
                                                           x_length:data_length})

            total_accuracy += accuracy
            total_loss += loss
            #print(i,accuracy,loss)

        total_accuracy = total_accuracy/i
        total_loss = total_loss/i
        print("Train set accuracy: %.2f %%"%(total_accuracy*100))
        print("Train set loss: %.2f "%(total_loss))
    

## Train my own word vectors

In [ ]:
    
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_corpus_words (integer): number of distinct words across the corpus
    """
    corpus_words = []
    num_corpus_words = -1
    
    # ------------------
    # Write your implementation here.
    flat_corpus = [y for x in corpus for y in x]
    for x in flat_corpus: 
        if x not in corpus_words: 
            corpus_words.append(x)
    corpus_words.sort()
    num_corpus_words = len(corpus_words)
    #print(corpus_words)
    # ------------------

    return corpus_words, num_corpus_words

def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).
    
        Note: Each word in a document should be at the center of a window. Words near edges will have a smaller
              number of co-occurring words.
              
              For example, if we take the document "START All that glitters is not gold END" with window size of 4,
              "All" will co-occur with "START", "that", "glitters", "is", and "not".
    
        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (numpy matrix of shape (number of corpus words, number of corpus words)): 
                Co-occurence matrix of word counts. 
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)
    print('Total %d distinct words in the corpus.'%num_words)
    M = None
    word2Ind = {}
    
    # ------------------
    # Write your implementation here.
    M = [[0 for i in range(num_words)] for j in range (num_words)]
    index = 0
    for w in words:
        word2Ind[w]=index
        index += 1
    for s in corpus:
        pos = 0
        for c in s:
            substr = []
            for i in range(1,window_size+1):                
                if pos-i >= 0 and s[pos-i] not in substr:
                    substr.append(s[pos-i])
                if pos+i <len(s) and s[pos+i] not in substr:           
                    substr.append(s[pos+i])
            for n in substr:
                M[word2Ind[c]][word2Ind[n]] += 1
            pos += 1
    # ------------------
    print("M shape", len(M), len(M[0]))
    M = np.array(M)
    return M, word2Ind

START_TOKEN = '<START>'
END_TOKEN = '<END>'
corpus = []

def reduce_to_k_dim(M, k=2):
    """ Reduce a co-occurence count matrix of dimensionality (num_corpus_words, num_corpus_words)
        to a matrix of dimensionality (num_corpus_words, k) using the following SVD function from Scikit-Learn:
            - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
    
        Params:
            M (numpy matrix of shape (number of corpus words, number of corpus words)): co-occurence matrix of word counts
            k (int): embedding size of each word after dimension reduction
        Return:
            M_reduced (numpy matrix of shape (number of corpus words, k)): matrix of k-dimensioal word embeddings.
                    In terms of the SVD from math class, this actually returns U * S
    """    
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Running Truncated SVD over %i words..." % (M.shape[0]))
    
    # ------------------
    # Write your implementation here.
    svd = TruncatedSVD(n_components=k)
    M_reduced=svd.fit_transform(M)
    # ------------------

    print("Done.")
    return M_reduced

for title in data.train_data:
    corpus.append([START_TOKEN] + title + [END_TOKEN])
matrix, word2ind = compute_co_occurrence_matrix(corpus, window_size=2)
print(matrix[0])
print(len(word2ind))

In [ ]:
import gensim
gensim.__version__

In [ ]:
import tensorflow as tf
tf.__version__